In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv()
logging.langsmith("Conversation_With_History")


LangSmith 추적을 시작합니다.
[프로젝트명]
Conversation_With_History


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser


In [3]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 Question-Answering 챗봇입니다. 주어진 질문에 대해 답변을 제공해주세요.",
        ),
        MessagesPlaceholder(variable_name = "chat_history"),
        (
            "human",
            "#Question:\n{question}"
        )
    ]
)
llm = ChatOpenAI(
    model_name = 'gpt-4o-mini',
    temperature= 0

)

chain = prompt | llm | StrOutputParser()

In [4]:
store ={}

def get_session_hisytory(session_id):
    print(f"[대화 세션 ID]:{session_id}")
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [5]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_hisytory,
    input_messages_key='question',
    history_messages_key="chat_history"

)

In [6]:
chain_with_history.invoke(
    {"question": "안녕 내 이름은 테디노트야"},
    config = {"configurable": {"session_id": "abc123"}}
)

[대화 세션 ID]:abc123


'안녕하세요, 테디노트님! 만나서 반갑습니다. 어떻게 도와드릴까요?'

In [7]:
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"},
    config = {"configurable": {"session_id": "abc123"}}
)

[대화 세션 ID]:abc123


'당신의 이름은 테디노트입니다!'

In [8]:
chain_with_history.invoke(
    {"question": "내 이름이 뭐라고?"},
    config = {"configurable": {"session_id": "abc1234"}}
)

[대화 세션 ID]:abc1234


'죄송하지만, 당신의 이름을 알 수 있는 정보가 없습니다. 당신의 이름을 알려주시면 그에 맞춰 대화할 수 있습니다!'